In [7]:
import xgboost as xgb 
import pandas as pd 
import numpy as np
import os
from tqdm import tqdm

In [8]:
bid_model = xgb.XGBClassifier()
bid_model.load_model("../model_weight/bid_using.model")
ask_model = xgb.XGBClassifier()
ask_model.load_model("../model_weight/ask_using.model")

In [18]:
PATH = r"../stock_dataset/2330"
DATE = "20221230"

FILE_PATH = os.path.join(PATH,DATE+".csv")
data = pd.read_csv(FILE_PATH)


HOLD_CAPITAL = 10000
HOLD_STOCK = 0

# buy = 0 , sell = 1 ,nothing = -1
predict_action = -1 
want_price = -1
tick_gap =0.5

loop = tqdm(range(len(data)-1))
for idx in loop :
    
    tick = data.iloc[idx].to_numpy()
    
    cur_match_price = tick[2]

    if cur_match_price == want_price:

        # buy
        if predict_action == 0 :

            HOLD_STOCK = HOLD_CAPITAL/cur_match_price
            HOLD_CAPITAL = 0
        # sell 
        elif predict_action == 1:
            HOLD_CAPITAL = HOLD_STOCK*cur_match_price
            HOLD_STOCK = 0 


    input_data = [[tick[3]]]    
    input_data[0].extend(tick[11:16])
    input_data[0].extend(tick[21:26])

    if HOLD_CAPITAL == 0:
        output = bid_model.predict_proba(input_data)
        choose = np.argmax(output[0])
        
        if choose == 1 :
            predict_action= 1
            want_price = cur_match_price+ tick_gap
        else :
            want_price = -1
            predict_action = -1
        print(f"bid {output[0][0]:%} {output[0][1]:%} {cur_match_price<data.iloc[idx+1].to_numpy()[2]}")
    elif HOLD_STOCK == 0:
        output = ask_model.predict_proba(input_data)
        choose = np.argmax(output[0])

        if choose == 1 :
            predict_action= 0
            want_price = cur_match_price -  tick_gap
        else :
            want_price = -1 
            predict_action = -1

        print(f"ask {output[0][0]:%} {output[0][1]:%} { cur_match_price > data.iloc[idx+1].to_numpy()[2]}" )
    loop.set_description(f"[{idx}/{len(data)}] {DATE} , capital: {HOLD_CAPITAL} , stock: {HOLD_STOCK}")
    loop.update
    if idx == len(data)-1:
        final_profit =HOLD_CAPITAL+HOLD_STOCK*cur_match_price
        print(f"final profit: {final_profit} , ROI: {final_profit/10000:2%}  ")
    




[5/5361] 20221230 , capital: 10011.00110011001 , stock: 0:   0%|          | 3/5360 [00:00<03:31, 25.30it/s]

ask 99.999923% 0.000075% False
ask 0.094765% 99.905235% True
bid 100.000000% 0.000000% False
bid 0.111455% 99.888545% True
ask 100.000000% 0.000000% False
ask 100.000000% 0.000000% False


[10/5361] 20221230 , capital: 0 , stock: 22.026405060748097:   0%|          | 9/5360 [00:00<03:38, 24.49it/s]

ask 100.000000% 0.000000% False
ask 0.000393% 99.999607% True
bid 100.000000% 0.000000% False
bid 100.000000% 0.000000% False
bid 0.000000% 100.000000% True


[15/5361] 20221230 , capital: 10022.014302640384 , stock: 0:   0%|          | 15/5360 [00:00<03:52, 23.02it/s]

...

[275/5361] 20221230 , capital: 10611.226909246372 , stock: 0:   5%|▌         | 275/5360 [00:10<03:25, 24.71it/s]

ask 99.791807% 0.208191% False
ask 100.000000% 0.000000% False
ask 100.000000% 0.000000% False
ask 99.999851% 0.000149% False
ask 99.961692% 0.038307% False


[280/5361] 20221230 , capital: 10622.874798279574 , stock: 0:   5%|▌         | 281/5360 [00:10<03:18, 25.62it/s]

ask 99.696535% 0.303467% False
ask 0.006294% 99.993706% True
bid 0.000000% 100.000000% True
ask 99.999988% 0.000013% False
ask 99.999994% 0.000009% False


KeyboardInterrupt: 